<a href="https://colab.research.google.com/github/mshinohar/langchain-book/blob/main/udemy_langchain_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Udemy講座「LangChainによる大規模言語モデル（LLM）アプリケーション開発入門」セクション「（アップデート）OpenAI の Chat API の Function calling 機能について」のソースコード

## 1. Function calling の基本

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

In [4]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [5]:
!pip install --quiet langchain==0.0.229 openai==0.27.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


以下はOpenAIの公式のサンプルコードをもとに一部変更（printの追加など）したソースコードです。

参考：https://platform.openai.com/docs/guides/gpt/function-calling

In [6]:
import openai
import json


# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    print("=== first response ===")
    print(response)
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        print("=== python function output ===")
        print(function_response)

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        print("=== second request messages ===")
        print(json.dumps(messages, indent=2))
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        print("=== second response ===")
        print(second_response)


run_conversation()

=== first response ===
{
  "id": "chatcmpl-8XSDekccDiZPWWvcAr7lj8o8EWq6I",
  "object": "chat.completion",
  "created": 1702983990,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  },
  "system_fingerprint": null
}
=== python function output ===
{"location": "Boston, MA", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}
=== second request messages ===
[
  {
    "role": "user",
    "content": "What's the weather like in Boston?"
  },
  {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "get_current_weather",
      "arguments": "{\n 

## 2. Function calling を使った LangChain の「OpenAI Functions Agent」

In [7]:
import json

import openai
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

tools = [
    Tool(
        name="get_current_weather",
        func=get_current_weather,
        description="Get the current weather in a given location"
    )
]

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = initialize_agent(tools, chat, agent=AgentType.OPENAI_FUNCTIONS)

result = agent.run("What's the weather like in Boston?")
print(result)


The current weather in Boston is 72 degrees Fahrenheit. It is sunny and windy.


In [8]:
import openai
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal", "llm-math"], llm=chat)
agent = initialize_agent(tools, chat, agent=AgentType.OPENAI_FUNCTIONS)

result = agent.run("Show files in ./sample_data directory.")
print(result)

/usr/local/lib/python3.10/dist-packages/langchain/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Here are the files in the `./sample_data` directory:

1. anscombe.json
2. california_housing_test.csv
3. california_housing_train.csv
4. mnist_test.csv
5. mnist_train_small.csv
6. README.md


## 3. Function calling を応用した Extraction と Tagging

以下はLangChainの公式ドキュメントのこちらのページを参考にしたサンプルコードです。

https://python.langchain.com/docs/modules/chains/additional/extraction

※ LangChainの公式ドキュメントは高い頻度でリンク切れになります。もしリンク切れになっている場合は、LangChainのドキュメントで「Extraction」と検索してみてください。

In [9]:
import json

import openai
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate

openai.log = "debug"
langchain.verbose = True

schema = {
    "properties": {
        "person_name": {"type": "string"},
        "person_height": {"type": "integer"},
        "person_hair_color": {"type": "string"},
        "dog_name": {"type": "string"},
        "dog_breed": {"type": "string"},
    },
    "required": ["person_name", "person_height"],
}
inp = """
Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
"""

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_extraction_chain(schema, chat)

result = chain.run(inp)
print(f"""=== result ===
{json.dumps(result, indent=2)}
===""")



> Entering new  chain...
Prompt after formatting:
Human: Extract and save the relevant entities mentioned in the following passage together with their properties.

Passage:

Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.




message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "Extract and save the relevant entities mentioned in the following passage together with their properties.\\n\\nPassage:\\n\\nAlex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.\\nAlex\'s dog Frosty is a labrador and likes to play hide and seek.\\n\\n"}], "model": "gpt-3.5-turbo", "max_tokens": null, "stream": false, "n": 1, "temperature": 0.0, "functions": [{"name": "information_extraction", "description": "Extracts the relevant information from the passage.", "parameters": {"type": "object", "properties": {"info": {"type": "array", "items": {"type": "object", "properties": {"person_name": {"title": "person_name", "type": "string"}, "person_height": {"title": "person_height", "type": "integer"}, "person_hair_color": {"title": "person_hair_color", "type": "string"}


> Finished chain.
=== result ===
[
  {
    "person_name": "Alex",
    "person_height": 5,
    "person_hair_color": "blonde",
    "dog_name": "Frosty",
    "dog_breed": "labrador"
  },
  {
    "person_name": "Claudia",
    "person_height": 6,
    "person_hair_color": "brunette"
  }
]
===


message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=3847 request_id=c1bfd066920b12da80f4bb8d0fb6042b response_code=200
body='{\n  "id": "chatcmpl-8XSGnPc9UplHqkG2yCQ1lGNyWQ2KX",\n  "object": "chat.completion",\n  "created": 1702984185,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": null,\n        "function_call": {\n          "name": "information_extraction",\n          "arguments": "{\\n  \\"info\\": [\\n    {\\n      \\"person_name\\": \\"Alex\\",\\n      \\"person_height\\": 5,\\n      \\"person_hair_color\\": \\"blonde\\",\\n      \\"dog_name\\": \\"Frosty\\",\\n      \\"dog_breed\\": \\"labrador\\"\\n    },\\n    {\\n      \\"person_name\\": \\"Claudia\\",\\n      \\"person_height\\": 6,\\n      \\"person_hair_color\\": \\"brunette\\"\\n    }\\n  ]\\n}"\n        }\n      },\n      "logprobs": null,\n      "finish_reason": "stop"\n    }\n  ],\n 

以下はLangChainの公式ドキュメントのこちらのページを参考にしたサンプルコードです。

https://python.langchain.com/docs/modules/chains/additional/tagging

※ LangChainの公式ドキュメントは高い頻度でリンク切れになります。もしリンク切れになっている場合は、LangChainのドキュメントで「Tagging」と検索してみてください。

In [10]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.prompts import ChatPromptTemplate

openai.log = "debug"
langchain.verbose = True

schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_tagging_chain(schema, chat)

inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"

result = chain.run(inp)
print(f"""=== result ===
{json.dumps(result, indent=2)}
===""")



> Entering new  chain...
Prompt after formatting:
Human: Extract the desired information from the following passage.

Passage:
Estoy muy enojado con vos! Te voy a dar tu merecido!



message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "Extract the desired information from the following passage.\\n\\nPassage:\\nEstoy muy enojado con vos! Te voy a dar tu merecido!\\n"}], "model": "gpt-3.5-turbo", "max_tokens": null, "stream": false, "n": 1, "temperature": 0.0, "functions": [{"name": "information_extraction", "description": "Extracts the relevant information from the passage.", "parameters": {"type": "object", "properties": {"sentiment": {"title": "sentiment", "type": "string"}, "aggressiveness": {"title": "aggressiveness", "type": "integer"}, "language": {"title": "language", "type": "string"}}, "required": []}}], "function_call": {"name": "information_extraction"}}' message='Post details'



> Finished chain.
=== result ===
{
  "sentiment": "enojado",
  "aggressiveness": 1,
  "language": "Spanish"
}
===


message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1702 request_id=9c5cac23f65cf2778328f23b0cc5e4a3 response_code=200
body='{\n  "id": "chatcmpl-8XSH38avAYyU8RDndAedJBP7b6nxw",\n  "object": "chat.completion",\n  "created": 1702984201,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": null,\n        "function_call": {\n          "name": "information_extraction",\n          "arguments": "{\\n  \\"sentiment\\": \\"enojado\\",\\n  \\"aggressiveness\\": 1,\\n  \\"language\\": \\"Spanish\\"\\n}"\n        }\n      },\n      "logprobs": null,\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 95,\n    "completion_tokens": 28,\n    "total_tokens": 123\n  },\n  "system_fingerprint": null\n}\n' headers='{\'Date\': \'Tue, 19 Dec 2023 11:10:02 GMT\', \'Content-Type\': \'application/json\', \'Transfer-Encoding\': \'chunked\', \'Connecti

## 4. 1 度に複数の関数を実行できる LangChain の「OpenAI Multi Functions Agent」

以下はLangChainの公式ドキュメントのこちらのページを参考にしたサンプルコードです。

https://python.langchain.com/docs/modules/agents/agent_types/openai_multi_functions_agent

※ LangChainの公式ドキュメントは高い頻度でリンク切れになります。もしリンク切れになっている場合は、LangChainのドキュメントで「OpenAI Multi Functions Agent」などと検索してみてください。

In [11]:
# duckduckgo-searchのバージョン3.8.3は動作しなくなったため、バージョン4.1.0をインストールします

!pip install --quiet duckduckgo-search==4.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 25.6 MB/s eta 0:00:00


In [12]:
import langchain
import openai
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

langchain.debug = True
langchain.verbose = False
openai.log = "info"

tools = load_tools(["ddg-search"])
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = initialize_agent(tools, chat, agent=AgentType.OPENAI_MULTI_FUNCTIONS)

result = agent.run(
    "What is the weather in LA and SF?"
)
print(f"""=== result ===
{result}
===""")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the weather in LA and SF?"
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI assistant.\nHuman: What is the weather in LA and SF?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [3.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": null,
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "name": "tool_selection",
                "arguments": "{\n  \"actions\": [\n    {\n      \"action_name\": \"duckduckgo_search\",\n      \"action\": {\n        \"query\": \"we